In [65]:
import json
import ipysheet as sheet
# !pip install ipywidgets
# !pip install ipysheet

In [155]:
def get_team_hp(team, units):
    return sum(u['hp'] for u in units if u['team'] == team)

def get_distance(u1, u2):
    return abs(u1['x'] - u2['x'])

def find_nearest_enemy(me, units):
    md, nearest = 9001, None
    for u in units:
        if u['hp'] <= 0 or u['team'] == me['team']:
            continue
        dx = get_distance(me, u)
        if dx < md:
            dx = md
            nearest = u
    return nearest

def battle(team1, team2, px1, px2, dt=0.1, logging=False):
    units = []
    for i, px, team in [[0, px1, team1], [1, px2, team2]]:
        for x, unit_params in zip(px, team):
            u = {k:v for k, v in unit_params.items()}
            u['hp'] = u['maxHealth']
            u['team'] = i
            u['ready'] = 0
            u['x'] = x
            units.append(u)
    time = 0
    for _ in range(900):
        if get_team_hp(0, units) <= 0 or get_team_hp(1, units) <= 0:
            break
        for un in units:
            if un['hp'] <= 0 or un['ready'] > time:
                continue
            enemy = find_nearest_enemy(un, units)
            if not enemy:
                continue
            if get_distance(un, enemy) <= un['attackRange']:
                if logging:
                    print(f'T{time:0.1f}: {un["type"]} team {un["team"]} hits {enemy["type"]} for {un["attackDamage"]}')
                enemy['hp'] -= un['attackDamage']
                un['ready'] = time + un['attackCooldown']
            else:
                x = un['x']
                un['x'] = x + [1, -1][un['team']] * (un['maxSpeed'] * dt)
                if logging:
                    print(f'T{time:0.1f}: {un["type"]} team {un["team"]} moves from {x} to {un["x"]}')

        time += dt
    return time, units

In [242]:
with open("data/GiantsGatesUnitsParams.json") as datafile:
    data = json.load(datafile)

RANGE_POWER_MULT = 1.7
    
for k in data.keys():
    data[k]["type"] = k

param_names = [
    'type',
    'cost',
    'maxSpeed',
    'maxHealth',
    'attackDamage',
    'attackCooldown',
    'attackRange']

column_names = param_names + ['dps', 'power']
unit_names = list(data.keys())
unit_sheet = sheet.sheet(
    columns=len(column_names),
    rows=len(unit_names),
#     row_headers=unit_names,
    column_headers=column_names,
    numeric_format='0')
data_rows = {}
for i, name in enumerate(unit_names):
    unit_data = data[name]
    cell_type = sheet.cell(i, 0, unit_data['type'])
    cell_cost = sheet.cell(i, 1, unit_data['cost'])
    cell_speed = sheet.cell(i, 2, unit_data['maxSpeed'])
    cell_health = sheet.cell(i, 3, unit_data['maxHealth'])
    cell_damage = sheet.cell(i, 4, unit_data['attackDamage'])
    cell_cooldown = sheet.cell(i, 5, unit_data['attackCooldown'])
    cell_range = sheet.cell(i, 6, unit_data['attackRange'])
    dps = unit_data['attackDamage'] / unit_data['attackCooldown']
    power = dps * unit_data['maxHealth'] / 5
    if unit_data['attackRange'] > 10:
        power *= RANGE_POWER_MULT
    cell_dps = sheet.cell(i, 7, dps, read_only=True, background_color='#dddddd')
    cell_power = sheet.cell(i, 8, power, read_only=True, background_color='#dddddd')
    data_rows[name] = [cell_type, cell_cost, cell_speed, cell_health, cell_damage, cell_cooldown, cell_range]
    
    @sheet.calculation(inputs=[cell_damage, cell_cooldown], output=cell_dps)
    def update_dps(a, b):
        return a / b
    
    @sheet.calculation(inputs=[cell_health, cell_damage, cell_cooldown, cell_range], output=cell_power)
    def update_power(h, d, c, r):
        p = h * d / c
        if r > 10:
            p *= RANGE_POWER_MULT
        return p
    
    

def get_unit_data(unit_type):
    
    res = {}
    for i, p in enumerate(param_names):
        res[p] = data_rows[unit_type][i].value
    return res

unit_sheet

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value='munchkin'), Cell(c…

In [249]:
munchkin = get_unit_data('munchkin')
thrower = get_unit_data('thrower')
ogre = get_unit_data('ogre')
shaman = get_unit_data('shaman')
brawler = get_unit_data('brawler')
warlock = get_unit_data('warlock')
headhunter = get_unit_data('headhunter')

t, result = battle([munchkin, munchkin], [shaman], [0, 0, 0], [10], logging=False)

print(f'Time: {t}')
for un in result:
    print(f'{un["type"]} from team {un["team"]} health is {un["hp"]}')
    

Time: 3.800000000000002
munchkin from team 0 health is 50
munchkin from team 0 health is 0
shaman from team 1 health is 0


In [233]:
for n1 in unit_names:
    for n2 in unit_names:
        unit1 = get_unit_data(n1)
        unit2 = get_unit_data(n2)
        t, result = battle([unit1], [unit2], [0], [20], logging=False)
        t = round(t, 2)
        hp1 = result[0]["hp"] / result[0]["maxHealth"]
        hp2 = result[1]["hp"] / result[1]["maxHealth"]
        print(f'Battle between {n1:>10} and {n2:<10} ended in {t} seconds. {hp1} : {hp2}')

Battle between   munchkin and munchkin   ended in 5.7 seconds. 0.0 : 0.1
Battle between   munchkin and thrower    ended in 3.8 seconds. 0.25 : 0.0
Battle between   munchkin and ogre       ended in 2.8 seconds. 0.0 : 0.8
Battle between   munchkin and shaman     ended in 2.2 seconds. 0.0 : 0.9
Battle between   munchkin and brawler    ended in 1.4 seconds. 0.0 : 0.975
Battle between   munchkin and warlock    ended in 1.5 seconds. -0.35 : 1.0
Battle between   munchkin and headhunter ended in 0.9 seconds. 0.0 : 1.0
Battle between    thrower and munchkin   ended in 3.8 seconds. 0.0 : 0.25
Battle between    thrower and thrower    ended in 2.5 seconds. 0.1 : -0.2
Battle between    thrower and ogre       ended in 2.6 seconds. -0.2 : 0.7
Battle between    thrower and shaman     ended in 0.8 seconds. 0.0 : 0.85
Battle between    thrower and brawler    ended in 2.0 seconds. 0.0 : 0.8875
Battle between    thrower and warlock    ended in 0.8 seconds. -0.8 : 0.85
Battle between    thrower and headhun